In [1]:
# FUNCIÓN  LAMBDA

cuadrado = lambda x: x ** 2
print(cuadrado(18))

324


In [6]:
"""
Contenido:
1) Gestión de inventario con *args
2) Cálculo de salarios con horas extra
3) Tres funciones tradicionales -> lambdas (con tipado en comentario)
4) Recursividad para “aplanar” listas

Cada sección incluye:
- Función(es) con tipado y docstring
- Ejemplos de uso con print()
- Comentarios paso a paso
"""
from __future__ import annotations
from typing import Any, Callable, Dict, Iterable, List, Tuple

# ================================
# 1) GESTIÓN DE INVENTARIO
# ================================
# Especificación solicitada:
# - gestionar_inventario(inventario: dict, *operaciones: tuple) -> dict
# - *args para aplicar varias tuplas de operaciones.
# - Lógica para agregar, actualizar y eliminar claves según cantidad <= 0.
# - Docstring con descripción, parámetros y retorno (anotaciones de tipo).
# - imprimir_inventario(inventario: dict) -> None

Operacion = Tuple[str, str, int] | Tuple[str, str]
# Formato permitido de operaciones:
# ("agregar", nombre, cantidad_delta)
# ("actualizar", nombre, cantidad_nueva)
# ("eliminar", nombre)


def gestionar_inventario(inventario: Dict[str, int], *operaciones: Operacion) -> Dict[str, int]:
    """Aplica múltiples operaciones sobre el inventario.

    Parámetros
    ----------
    inventario : Dict[str, int]
        Diccionario {producto: cantidad} que se modificará en el lugar.
    *operaciones : Operacion
        Tuplas con la forma:
        - ("agregar", nombre, cantidad_delta): suma/resta unidades (puede ser negativo)
        - ("actualizar", nombre, cantidad_nueva): fija la cantidad exacta
        - ("eliminar", nombre): elimina la clave del inventario

    Reglas
    ------
    - Si, tras "agregar" o "actualizar", la cantidad es <= 0, la clave se elimina.
    - Las operaciones se aplican en el orden provisto.

    Retorno
    -------
    Dict[str, int]
        El mismo diccionario `inventario` (modificado), para permitir encadenamiento.
    """
    for op in operaciones:
        if not op:
            continue
        accion = op[0].lower()

        if accion == "eliminar":
            # op = ("eliminar", nombre)
            if len(op) != 2:
                raise ValueError("Operación 'eliminar' debe ser (eliminar, nombre)")
            nombre = op[1]
            inventario.pop(nombre, None)  # no falla si no existe

        elif accion == "agregar":
            # op = ("agregar", nombre, cantidad_delta)
            if len(op) != 3:
                raise ValueError("Operación 'agregar' debe ser (agregar, nombre, cantidad_delta)")
            nombre, delta = op[1], int(op[2])
            actual = inventario.get(nombre, 0) + delta
            if actual <= 0:
                inventario.pop(nombre, None)
            else:
                inventario[nombre] = actual

        elif accion == "actualizar":
            # op = ("actualizar", nombre, cantidad_nueva)
            if len(op) != 3:
                raise ValueError("Operación 'actualizar' debe ser (actualizar, nombre, cantidad_nueva)")
            nombre, nueva = op[1], int(op[2])
            if nueva <= 0:
                inventario.pop(nombre, None)
            else:
                inventario[nombre] = nueva
        else:
            raise ValueError(f"Acción no soportada: {accion!r}")

    return inventario


def imprimir_inventario(inventario: Dict[str, int]) -> None:
    """Imprime las claves y cantidades del inventario, ordenadas alfabéticamente."""
    if not inventario:
        print("Inventario vacío.")
        return
    print("Inventario final:")
    for nombre in sorted(inventario):
        print(f"- {nombre}: {inventario[nombre]}")


# --- Ejemplo de uso (1) ---
if __name__ == "__main__":
    inv = {"manzanas": 10, "bananas": 5}
    gestionar_inventario(
        inv,
        ("agregar", "manzanas", 5),       # manzanas -> 15
        ("agregar", "peras", 3),           # peras -> 3 (nueva)
        ("actualizar", "bananas", 0),      # 0 => se elimina "bananas"
        ("eliminar", "inexistente"),       # no falla
        ("agregar", "peras", -3),          # peras -> 0 => se elimina
    )
    imprimir_inventario(inv)



Inventario final:
- manzanas: 15


In [36]:
# Funcion 1: Gestionar Inventario #CORRECCIÓN EN CLASE
inventario = {'manzanas':10, 'naranjas':5}
def gestionar_inventario(inventario: Dict, *actualización: tuple) -> Dict:
    for producto, cantidad in actualización:
        if producto in inventario:
            inventario[producto] += cantidad
            if inventario <= 0:
                del inventario[producto]
        else:
            if cantidad > 0:
                inventario[producto] = cantidad

print(inventario)

{'manzanas': 10, 'naranjas': 5}


In [40]:
resultado = gestionar_inventario(inventario, ('pera',15), ('manzanas',-10), ('naranjas',8))

print(resultado)

TypeError: '<=' not supported between instances of 'dict' and 'int'

In [3]:
# ================================
# 2) CÁLCULO DE SALARIOS CON HORAS EXTRA
# ================================
# - calcular_salario(tarifa_hora: float, *horas_diarias: int) -> float
# - *args con 7 valores (puede validar)
# - Base hasta 40 h; resto a 1.5x
# - imprimir_reporte(...) que muestre desglose


def calcular_salario(tarifa_hora: float, *horas_diarias: int) -> float:
    """Calcula el salario semanal según horas y tarifa.

    Parámetros
    ----------
    tarifa_hora : float
        Tarifa por hora.
    *horas_diarias : int
        Horas trabajadas por día (idealmente 7 valores para una semana).

    Regla
    -----
    - Las primeras 40 horas se pagan a tarifa base.
    - Horas adicionales se pagan a 1.5 * tarifa.

    Retorno
    -------
    float
        Total a pagar.
    """
    if len(horas_diarias) != 7:
        raise ValueError("Debe proporcionar exactamente 7 valores de horas diarias.")

    total_horas = sum(int(h) for h in horas_diarias)
    horas_base = min(total_horas, 40)
    horas_extra = max(total_horas - 40, 0)

    salario_base = horas_base * tarifa_hora
    salario_extra = horas_extra * tarifa_hora * 1.5

    return salario_base + salario_extra


def imprimir_reporte(tarifa_hora: float, *horas_diarias: int) -> None:
    """Imprime un reporte con el desglose de horas y pagos."""
    total_horas = sum(int(h) for h in horas_diarias)
    horas_base = min(total_horas, 40)
    horas_extra = max(total_horas - 40, 0)
    salario_base = horas_base * tarifa_hora
    salario_extra = horas_extra * tarifa_hora * 1.5
    total = salario_base + salario_extra

    print("Reporte semanal de horas y salario")
    print("-" * 34)
    print(f"Horas totales : {total_horas}")
    print(f"Horas base    : {horas_base}")
    print(f"Horas extra   : {horas_extra}")
    print(f"Salario base  : {salario_base:.2f}")
    print(f"Salario extra : {salario_extra:.2f}")
    print(f"TOTAL         : {total:.2f}")


# --- Ejemplo de uso (2) ---
if __name__ == "__main__":
    imprimir_reporte(10.0, 8, 8, 8, 8, 8, 0, 5)  # 45h totales -> 40 base + 5 extra

Reporte semanal de horas y salario
----------------------------------
Horas totales : 45
Horas base    : 40
Horas extra   : 5
Salario base  : 400.00
Salario extra : 75.00
TOTAL         : 475.00


In [2]:

# ================================
# 3) TRADICIONALES -> LAMBDAS
# ================================
# - Mantener firma tipada mediante comentario # type: Callable[...] en la lambda
# - Incluir docstring en la versión clásica

from math import fsum


def promedio(valores: Iterable[float]) -> float:
    """Devuelve el promedio aritmético de una colección de números.

    Usa `fsum` para mejorar la estabilidad numérica al sumar floats.
    """
    vals = list(valores)
    if not vals:
        raise ValueError("No se puede promediar una colección vacía.")
    return fsum(vals) / len(vals)

# Versión lambda equivalente
promedio_lambda = lambda valores: (lambda vs: (_ for _ in ()).throw(ValueError("No se puede promediar una colección vacía.")) if not vs else fsum(vs) / len(vs))(list(valores))  # type: Callable[[Iterable[float]], float]



def suma_cuadrados(valores: Iterable[float]) -> float:
    """Suma los cuadrados de los valores provistos."""
    return fsum((x * x) for x in valores)

# Versión lambda equivalente
suma_cuadrados_lambda = lambda valores: fsum((x * x) for x in valores)  # type: Callable[[Iterable[float]], float]



def porcentaje_cambio(valor_inicial: float, valor_final: float) -> float:
    """Calcula el % de cambio entre `valor_inicial` y `valor_final`.

    Fórmula: ((valor_final - valor_inicial) / valor_inicial) * 100
    """
    if valor_inicial == 0:
        raise ZeroDivisionError("valor_inicial no puede ser 0")
    return ((valor_final - valor_inicial) / valor_inicial) * 100

# Versión lambda equivalente
porcentaje_cambio_lambda = (
    lambda vi, vf: (_ for _ in ()).throw(ZeroDivisionError("valor_inicial no puede ser 0")) if vi == 0 else ((vf - vi) / vi) * 100  # type: Callable[[float, float], float]
)


# --- Ejemplos de uso (3) ---
if __name__ == "__main__":
    datos = [1.0, 2.0, 3.0, 4.0]
    print("promedio:", promedio(datos))
    print("promedio_lambda:", promedio_lambda(datos))

    print("suma_cuadrados:", suma_cuadrados(datos))
    print("suma_cuadrados_lambda:", suma_cuadrados_lambda(datos))

    print("porcentaje_cambio 100->130:", porcentaje_cambio(100, 130))
    print("porcentaje_cambio_lambda 100->130:", porcentaje_cambio_lambda(100, 130))


promedio: 2.5
promedio_lambda: 2.5
suma_cuadrados: 30.0
suma_cuadrados_lambda: 30.0
porcentaje_cambio 100->130: 30.0
porcentaje_cambio_lambda 100->130: 30.0


In [1]:

# ================================
# 4) RECURSIVIDAD: APLANAR LISTA
# ================================
# - aplanar_lista(datos: list) -> list
# - Detecta sublistas y concatena resultados recursivos


def aplanar_lista(datos: List[Any]) -> List[Any]:
    """Devuelve una versión plana de una lista potencialmente anidada.

    Ejemplos:
    >>> aplanar_lista([1, [2, 3], 4])
    [1, 2, 3, 4]
    >>> aplanar_lista([[1, [2, [3]]], 4])
    [1, 2, 3, 4]
    """
    resultado: List[Any] = []

    for elemento in datos:
        if isinstance(elemento, list):
            # Caso recursivo: extendemos con el resultado plano de la sublista
            resultado.extend(aplanar_lista(elemento))
        else:
            # Caso base: no es lista -> lo agregamos tal cual
            resultado.append(elemento)

    return resultado


# --- Ejemplo de uso (4) ---
if __name__ == "__main__":
    anidada = [1, [2, [3, 4], []], [5, [6, [7]]]]
    print("Lista anidada:", anidada)
    print("Lista aplanada:", aplanar_lista(anidada))


Lista anidada: [1, [2, [3, 4], []], [5, [6, [7]]]]
Lista aplanada: [1, 2, 3, 4, 5, 6, 7]
